# [Module 4.1] Deploy Built-in TFS Image


여기서는 다음과 같은 작업을 합니다.

- 모델 아티펙트 (model.tar.gz) 파일을 S3에서 로컬에 다운로드
- TF Saved_Model 의 정의를 확인
- SageMaker Model 생성
- Endpoint 생성
- Inference의 Request Serializer and Deserializer 생성
- 프리딕터 생성
- 셈플 데이타로 추론

---
이 노트북은 약 10분 정도 소요 됩니다.


필요한 프로그램 설치

In [17]:
!pip install -q --upgrade pip
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==2.1.0
!pip install -q transformers==2.8.0
!pip install -q sagemaker==1.56.1

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

astroid 2.3.3 requires wrapt==1.11.*, but you'll have wrapt 1.12.1 which is incompatible.


In [18]:
import boto3
import sagemaker
import pandas as pd
import tensorflow as tf
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [19]:
%store -r training_job_name

In [20]:
print(training_job_name)

bert2tweet-2020-08-19-05-23-18-129


## 모델 정의 확인

아래의 세개의 셀은 모델을 로컬에 다운로드 받고 텐서플로우의 Saved Model의 정의 (입력, 출력)를 확인하는 코드 입니다. 확인을 원하시면 아래 주석을 제거하고 실행하시면 됩니다.

In [21]:
# model_download = 'model'
# os.makedirs(model_download, exist_ok=True)

In [22]:
# !aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz {model_download}/model.tar.gz

In [23]:
# !tar -xvzf   {model_download}/model.tar.gz
# !saved_model_cli show --all --dir ./tensorflow/saved_model/0/

## SageMaker Model 생성

In [24]:
import os
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              framework_version='2.0.0') # Elastic Inference does not yet support TF 2.1.0 as of sagemaker==1.56.1

## Endpoint 생성

In [25]:
instance_type='ml.m4.xlarge'

import time
endpont_name    = f'builtin-inference-image-endpoint-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'



deployed_model = model.deploy(
                             endpoint_name = endpont_name,
                             initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)



-------------!

In [26]:
endpoint_name = deployed_model.endpoint
print('Endpoint name:  {}'.format(endpoint_name))

Endpoint name:  builtin-inference-image-endpoint-2020-08-19-05-54-43


## Inference Request Serializer and Deserializer 생성

In [27]:
class RequestHandler(object):
    import json
    
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __call__(self, instances):
        transformed_instances = []

        for instance in instances:
            encode_plus_tokens = tokenizer.encode_plus(instance,
                                                       pad_to_max_length=True,
                                                       max_length=self.max_seq_length)

            input_ids = encode_plus_tokens['input_ids']
            input_mask = encode_plus_tokens['attention_mask']
            segment_ids = [0] * self.max_seq_length

            transformed_instance = {"input_ids": input_ids, 
                                    "input_mask": input_mask, 
                                    "segment_ids": segment_ids}

            transformed_instances.append(transformed_instance)

        transformed_data = {"instances": transformed_instances}

        return json.dumps(transformed_data)
    
class ResponseHandler(object):
    import json
    import tensorflow as tf
    
    def __init__(self, classes):
        self.classes = classes
    
    def __call__(self, response, accept_header):
        import tensorflow as tf

        response_body = response.read().decode('utf-8')

        response_json = json.loads(response_body)

        log_probabilities = response_json["predictions"]

#        predicted_classes = []

        # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
#         for log_probability in log_probabilities:
#             softmax = tf.nn.softmax(log_probability)    
#             predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
#             predicted_class = self.classes[predicted_class_idx]
#             predicted_classes.append(predicted_class)

        return log_probabilities    

## Predictor 생성

In [28]:
import json
from sagemaker.tensorflow.serving import Predictor
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

request_handler = RequestHandler(tokenizer=tokenizer,
                                 max_seq_length=32)

response_handler = ResponseHandler(classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sess,
                      serializer=request_handler,
                      deserializer=response_handler,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

## Inference 실행

In [29]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [30]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
2074,lawa siot wendy,5
1001,finished all 5 haunted houses in 2 hours,1
3950,daddy,7
1882,you guys will also perform these steps,5
5257,check out me new video cut retweet comment 1u...,2
1400,rare moment between mannu salahuddin sannu ma...,5
5372,holy wow,5
7487,s'talk loan agreements,9
6391,how cute is evelyne holding cosima's funko pop,5
3978,definitely,2


In [31]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


In [32]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 5
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]
    
#     print("reviews: \n", reviews)



    predicted_classes = predictor.predict(reviews)[0]
#    predicted_classes = predictor.predict(reviews)    
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('-------------------------------------------')    
    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    

    print('Prediction: {},{},{},{},{},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2]),                                       
        predicted_classes[3], 
        tweet_util.get_emo_class_label(predicted_classes[3]),                                      
        predicted_classes[4], 
        tweet_util.get_emo_class_label(predicted_classes[4]),                                      
        
        ))    
        



-------------------------------------------
tweet: lawa siot wendy 
Ground_truth- 5:😍
 
Prediction: 3,😂,5,😍,2,🔥,1,💕,7,😭 
 
-------------------------------------------
tweet:  finished all 5 haunted houses in 2 hours  
Ground_truth- 1:💕
 
Prediction: 5,😍,2,🔥,8,🙄,4,😊,6,😩 
 
-------------------------------------------
tweet:  daddy 
Ground_truth- 7:😭
 
Prediction: 5,😍,1,💕,3,😂,7,😭,4,😊 
 
-------------------------------------------
tweet:  you guys will also perform these steps  
Ground_truth- 5:😍
 
Prediction: 2,🔥,1,💕,3,😂,5,😍,4,😊 
 
-------------------------------------------
tweet:  check out me new video cut retweet comment 1up westoakland realrichmond 
Ground_truth- 2:🔥
 
Prediction: 2,🔥,4,😊,1,💕,0,❤,5,😍 
 
-------------------------------------------
tweet:  rare moment between mannu salahuddin sannu mannu mannmayal 5daystogo lastepisode humtv mayaali https t  
Ground_truth- 5:😍
 
Prediction: 5,😍,3,😂,0,❤,4,😊,7,😭 
 
-------------------------------------------
tweet:  holy wow 
Ground_trut